Paper Trading port: 7497
Live Trading port: 7496

In [1]:
from ib_insync import *

# First, disconnect any existing connections
try:
    ib.disconnect()
except:
    pass

# Start fresh
util.startLoop()
ib = IB()

# Try connecting with a different client ID
try:
    ib.connect('127.0.0.1', 7496, clientId=123) 
    print("Connected successfully")
    
    # Create Apple contract
    apple = Stock('AAPL', 'SMART', 'USD')

    # Request market data
    ticker = ib.reqMktData(apple)

    # Wait a moment for data to arrive
    ib.sleep(3)

    # Print the data
    print(f'AAPL Price Data:')
    print(f'Bid: {ticker.bid}')
    print(f'Ask: {ticker.ask}')
    print(f'Last: {ticker.last}')
    
except Exception as e:
    print(f"Error: {str(e)}")
    
finally:
    if ib.isConnected():
        ib.disconnect()
        print("Disconnected")

API connection failed: ConnectionRefusedError(61, "Connect call failed ('127.0.0.1', 7496)")
Make sure API port on TWS/IBG is open


Error: [Errno 61] Connect call failed ('127.0.0.1', 7496)


In [4]:
from ib_insync import *

# Connect
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7496, clientId=123)

try:
    # Create Apple contract
    apple = Stock('AAPL', 'SMART', 'USD')
    
    # Get historical data (last 5 days, 1 hour bars)
    bars = ib.reqHistoricalData(
        apple,
        endDateTime='',  # empty means now
        durationStr='5 D',  # for last 5 days
        barSizeSetting='1 hour',
        whatToShow='TRADES',
        useRTH=True  # Regular Trading Hours only
    )
    
    # Convert to pandas DataFrame and print
    df = util.df(bars)
    print(df)

finally:
    ib.disconnect()
    

                        date    open    high     low   close     volume  \
0  2024-12-31 09:30:00-05:00  252.44  253.28  251.55  252.27  2668812.0   
1  2024-12-31 10:00:00-05:00  252.25  252.40  251.31  251.82  2135462.0   
2  2024-12-31 11:00:00-05:00  251.83  251.85  251.02  251.32  1574747.0   
3  2024-12-31 12:00:00-05:00  251.32  251.53  250.06  250.67  2242562.0   
4  2024-12-31 13:00:00-05:00  250.66  250.74  249.43  249.88  2095024.0   
5  2024-12-31 14:00:00-05:00  249.89  250.43  249.67  250.42  1883135.0   
6  2024-12-31 15:00:00-05:00  250.41  251.08  249.50  250.38  6301045.0   
7  2025-01-02 09:30:00-05:00  248.93  249.10  244.80  246.85  7247563.0   
8  2025-01-02 10:00:00-05:00  246.89  247.81  244.82  245.42  5857237.0   
9  2025-01-02 11:00:00-05:00  245.42  245.49  244.16  244.64  3651814.0   
10 2025-01-02 12:00:00-05:00  244.68  245.25  242.31  242.99  4403950.0   
11 2025-01-02 13:00:00-05:00  242.98  243.03  241.82  242.23  3807049.0   
12 2025-01-02 14:00:00-05

### Fetch Benzinga Pro HeadLines News

In [13]:
from ib_insync import *
from datetime import datetime

# First, disconnect any existing connections
try:
    ib.disconnect()
except:
    pass

# Start fresh
util.startLoop()
ib = IB()

try:
    ib.connect('127.0.0.1', 7496, clientId=456)
    print("Connected successfully")
    
    # Create Apple contract
    apple = Stock('AAPL', 'SMART', 'USD')
    
    # 1. Historical Benzinga News
    details = ib.reqContractDetails(apple)
    if details:
        conId = details[0].contract.conId
        print(f"Got contract ID: {conId}")
        
        # Request historical news with specific date
        news = ib.reqHistoricalNews(
            conId,          # conId
            "BZ",          # providerCodes
            "2024-01-20",  # startDateTime (YYYY-MM-DD format)
            "2024-01-21",  # endDateTime
            10,            # totalResults
            []             # historicalNewsOptions
        )
        
        print("\nHistorical News:")
        for n in news:
            print(f"Time: {n.time}, Headline: {n.headline}")
            
    # 2. Live Benzinga News
    print("\nSubscribing to Live News...")
    news_contract = Contract()
    news_contract.symbol = "BZ:BZ_ALL"
    news_contract.secType = "NEWS"
    news_contract.exchange = "BZ"
    
    # Subscribe to live news
    ib.reqMktData(news_contract, "mdoff,292", False, False)
    
    # Wait for some data
    ib.sleep(5)
    
except Exception as e:
    print(f"Error: {str(e)}")
    
finally:
    if ib.isConnected():
        ib.disconnect()
        print("Disconnected")

Connected successfully
Got contract ID: 265598

Historical News:
Time: 2025-01-08 11:35:11, Headline: {A:800015:L:en:K:0.54:C:0.5353515148162842}Dell Simplifies PC Lineup With Apple-Like Branding: XPS, Inspiron Brands Are Gone For Good
Time: 2025-01-07 21:47:40, Headline: {A:800015:L:en:K:n/a:C:0.4840547442436218}!3 Digital Infrastructure ETFs In Focus As Trump Reveals Billion-Dollar Data Center Construction Plan
Time: 2025-01-07 18:39:43, Headline: {A:800015:L:en:K:-0.97:C:0.97}!Apple shares are trading lower after MoffettNathanson downgraded the stock from Neutral to Sell and lowered its price target from $202 to $188.
Time: 2025-01-07 16:36:19, Headline: {A:800015:L:en}!Apple Analyst Flags iPhone AI Struggles, Google Deal Risks
Time: 2025-01-07 16:00:56, Headline: {A:800015:L:en:K:n/a:C:0.9524242877960205}This Is What Whales Are Betting On Apple
Time: 2025-01-07 15:00:32, Headline: {A:800015:L:en}Evaluating Apple Against Peers In Technology Hardware, Storage &amp; Peripherals Indust

In [15]:
from ib_insync import *
from datetime import datetime

# Connect
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7496, clientId=456)

try:
    # Create Apple contract
    apple = Stock('AAPL', 'SMART', 'USD')
    
    # Get contract details
    details = ib.reqContractDetails(apple)
    if details:
        conId = details[0].contract.conId
        print(f"Got contract ID: {conId}")
        
        # Request historical news with specific date
        news = ib.reqHistoricalNews(
            conId,          # conId
            "BZ",          # providerCodes
            "2024-01-20",  # startDateTime (YYYY-MM-DD format)
            "2024-01-21",  # endDateTime
            10,            # totalResults
            []             # historicalNewsOptions
        )
        
        print("\nHistorical News:")
        for n in news:
            print(f"\nTime: {n.time}")
            print(f"Headline: {n.headline}")
            
            # Get full article for each headline
            article = ib.reqNewsArticle("BZ", n.articleId)
            print(f"Full Article Text: {article.articleText}\n")
            print("-" * 50)
            
finally:
    ib.disconnect()
    print("Disconnected")

Got contract ID: 265598

Historical News:

Time: 2025-01-08 11:35:11
Headline: {A:800015:L:en:K:0.54:C:0.5353515148162842}Dell Simplifies PC Lineup With Apple-Like Branding: XPS, Inspiron Brands Are Gone For Good
Full Article Text: <p>On Monday,<strong> Dell Technologies Inc.</strong> (NYSE:<a class="ticker" href="https://www.benzinga.com/stock/DELL#NYSE">DELL</a>) announced a major rebranding of its personal computers, aiming to boost sales by adopting a simplified naming strategy similar to <strong>Apple Inc.</strong> (NASDAQ:<a class="ticker" href="https://www.benzinga.com/stock/AAPL#NASDAQ">AAPL</a>).</p>&#10;&#10;&#10;&#10;<p><strong>What Happened:</strong> The company has decided to retire long-standing product names like &#8220;XPS&#8221; and &#8220;Inspiron&#8221; in favor of a streamlined approach centered around the Dell name.</p>&#10;&#10;&#10;&#10;<p>The rebranding is part of Dell&#8217;s strategy to make its products more appealing and less confusing for consumers. The new

In [16]:
from ib_insync import *
import pandas as pd
from datetime import datetime
import time

# Connect
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7496, clientId=456)

try:
    # Create news contract for all Benzinga news
    news_contract = Contract()
    news_contract.symbol = "BZ:BZ_ALL"
    news_contract.secType = "NEWS"
    news_contract.exchange = "BZ"
    
    # Get contract details
    details = ib.reqContractDetails(news_contract)
    if details:
        conId = details[0].contract.conId
        print(f"Got contract ID: {conId}")
        
        # Request historical news
        news = ib.reqHistoricalNews(
            conId,          
            "BZ",          
            "2024-01-20",  # Start date
            "2024-01-21",  # End date
            1000,          # Get more results
            []
        )
        
        # Create lists to store data
        news_data = []
        
        print("\nFetching news and articles...")
        for n in news:
            try:
                # Get full article
                article = ib.reqNewsArticle("BZ", n.articleId)
                
                # Store all data
                news_item = {
                    'timestamp': n.time,
                    'headline': n.headline,
                    'article_id': n.articleId,
                    'article_text': article.articleText,
                    'symbols': article.articleText.split('STOCK TICKERS: ')[-1].split('\n')[0] if 'STOCK TICKERS: ' in article.articleText else '',
                    'source': 'Benzinga Pro'
                }
                
                news_data.append(news_item)
                print(f"Processed article: {n.articleId}")
                
                # Small delay to avoid rate limits
                time.sleep(0.1)
                
            except Exception as e:
                print(f"Error processing article {n.articleId}: {str(e)}")
        
        # Create DataFrame
        df = pd.DataFrame(news_data)
        
        # Clean and format
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.sort_values('timestamp', ascending=False)
        
        # Save to CSV
        df.to_csv('benzinga_news.csv', index=False)
        print("\nData saved to benzinga_news.csv")
        
        # Display sample
        print("\nSample of collected news:")
        print(df.head())
        
finally:
    ib.disconnect()
    print("\nDisconnected")

Got contract ID: 137116869

Fetching news and articles...
Processed article: BZ$19b6d200
Processed article: BZ$19b6d1f9
Processed article: BZ$19b6d120
Processed article: BZ$19b6d089
Processed article: BZ$19b6d00f
Processed article: BZ$19b6cfa7
Processed article: BZ$19b6cf35
Processed article: BZ$19b6ce63
Processed article: BZ$19b6cdb5
Processed article: BZ$19b6cd7a
Processed article: BZ$19b6cd10
Processed article: BZ$19b6cc92
Processed article: BZ$19b6cc66
Processed article: BZ$19b6cc64
Processed article: BZ$19b6cbaa
Processed article: BZ$19b6cb8a
Processed article: BZ$19b6cb77
Processed article: BZ$19b6cac1
Processed article: BZ$19b6cabe
Processed article: BZ$19b6c9e8
Processed article: BZ$19b6c97c
Processed article: BZ$19b6c961
Processed article: BZ$19b6c8b5
Processed article: BZ$19b6c86c
Processed article: BZ$19b6c850
Processed article: BZ$19b6c841
Processed article: BZ$19b6c777
Processed article: BZ$19b6c6e0
Processed article: BZ$19b6c6c7
Processed article: BZ$19b6c54d
Processed ar

In [19]:
from ib_insync import *
import pandas as pd
from datetime import datetime
import time
import re

# Connect
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7496, clientId=456)

try:
    # Create news contract for all Benzinga news
    news_contract = Contract()
    news_contract.symbol = "BZ:BZ_ALL"
    news_contract.secType = "NEWS"
    news_contract.exchange = "BZ"
    
    # Get contract details
    details = ib.reqContractDetails(news_contract)
    if details:
        conId = details[0].contract.conId
        print(f"Got contract ID: {conId}")
        
        # Request historical news
        news = ib.reqHistoricalNews(
            conId,          
            "BZ",          
            "2024-01-20",  # Start date
            "2024-01-21",  # End date
            1000,          # Get more results
            []
        )
        
        # Create lists to store data
        news_data = []
        
        def extract_symbols(text):
            # Try different patterns
            patterns = [
                r'STOCK TICKERS:\s*([^\n]+)',
                r'Stocks:\s*([^\n]+)',
                r'Ticker\(s\):\s*([^\n]+)',
                r'\$([A-Z]+)',  # Catches $AAPL style tickers
                r'\(NYSE:\s*([A-Z]+)\)',  # Catches (NYSE: AAPL) style
                r'\(NASDAQ:\s*([A-Z]+)\)'  # Catches (NASDAQ: AAPL) style
            ]
            
            symbols = []
            for pattern in patterns:
                matches = re.findall(pattern, text)
                if matches:
                    # Clean up the matches
                    for match in matches:
                        clean_symbols = re.findall(r'[A-Z]+', match)
                        symbols.extend(clean_symbols)
            
            return list(set(symbols))  # Remove duplicates
        
        print("\nFetching news and articles...")
        for n in news:
            try:
                # Get full article
                article = ib.reqNewsArticle("BZ", n.articleId)
                
                # Extract symbols using the new function
                symbols = extract_symbols(article.articleText)
                
                # Store all data
                news_item = {
                    'timestamp': n.time,
                    'headline': n.headline,
                    'article_id': n.articleId,
                    'article_text': article.articleText,
                    'symbols': ', '.join(symbols) if symbols else '',
                    'source': 'Benzinga Pro'
                }
                
                news_data.append(news_item)
                print(f"Processed article: {n.articleId} - Symbols: {news_item['symbols']}")
                
                # Small delay to avoid rate limits
                time.sleep(0.1)
                
            except Exception as e:
                print(f"Error processing article {n.articleId}: {str(e)}")
        
        # Create DataFrame
        df = pd.DataFrame(news_data)
        
        # Clean and format
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.sort_values('timestamp', ascending=False)
        
        # Save to CSV
        df.to_csv('benzinga_news.csv', index=False)
        print("\nData saved to benzinga_news.csv")
        
        # Display sample
        print("\nSample of collected news:")
        print(df[['timestamp', 'headline', 'symbols']].head())
        
finally:
    ib.disconnect()
    print("\nDisconnected")

Got contract ID: 137116869

Fetching news and articles...
Processed article: BZ$19b6d4e5 - Symbols: 
Processed article: BZ$19b6d4da - Symbols: 
Processed article: BZ$19b6d4d0 - Symbols: 
Processed article: BZ$19b6d4cc - Symbols: 
Processed article: BZ$19b6d4c2 - Symbols: 
Processed article: BZ$19b6d460 - Symbols: 
Processed article: BZ$19b6d39d - Symbols: 
Processed article: BZ$19b6d399 - Symbols: 
Processed article: BZ$19b6d38c - Symbols: 
Processed article: BZ$19b6d372 - Symbols: 
Processed article: BZ$19b6d245 - Symbols: 
Processed article: BZ$19b6d200 - Symbols: 
Processed article: BZ$19b6d1f9 - Symbols: 
Processed article: BZ$19b6d120 - Symbols: 
Processed article: BZ$19b6d089 - Symbols: 
Processed article: BZ$19b6d00f - Symbols: 
Processed article: BZ$19b6cfa7 - Symbols: 
Processed article: BZ$19b6cf35 - Symbols: 
Processed article: BZ$19b6ce63 - Symbols: 
Processed article: BZ$19b6cdb5 - Symbols: 
Processed article: BZ$19b6cd7a - Symbols: 
Processed article: BZ$19b6cd10 - Symbol

In [20]:
df.head()

,timestamp,headline,article_id,article_text,symbols,source
0,2025-01-08 12:57:16,{A:800015:L:en:K:0.97:C:0.97}!Northland Capita...,BZ$19b6d4e5,Northland Capital Markets analyst Ted Jackson...,,Benzinga Pro
1,2025-01-08 12:57:10,{A:800015:L:en:K:0.99:C:0.9908973574638367}!Al...,BZ$19b6d4da,<p>Copyright &#169; Benzinga. All rights reser...,,Benzinga Pro
2,2025-01-08 12:56:59,{A:800015:L:en:K:0.94:C:0.9400199055671692}!Ma...,BZ$19b6d4d0,<p>Copyright &#169; Benzinga. All rights reser...,,Benzinga Pro
3,2025-01-08 12:56:49,{A:800015:L:en:K:0.98:C:0.9794827699661255}!eB...,BZ$19b6d4cc,<p>Copyright &#169; Benzinga. All rights reser...,,Benzinga Pro
4,2025-01-08 12:56:48,{A:800015:L:en:K:0.97:C:0.97}!D. Boral Capital...,BZ$19b6d4c2,D. Boral Capital analyst Jesse Sobelson mai...,,Benzinga Pro


In [26]:
from ib_insync import *
import pandas as pd
from datetime import datetime
import re
import time

class NewsCollector:
    def __init__(self):
        self.ib = IB()
        self.news_data = []
        
    def connect(self):
        self.ib.connect('127.0.0.1', 7496, clientId=456)
        
    def extract_symbols(self, text):
        """Extract stock symbols from article text using multiple patterns"""
        symbols = set()
        patterns = [
            r'STOCK TICKERS:\s*([^.]*)',
            r'Ticker\(s\):\s*([^.]*)',
            r'(?:NYSE|NASDAQ):\s*([A-Z]+)',
            r'\$([A-Z]{1,5})\b',
            r'\b[A-Z]{1,5}\b(?=\s*(?:shares|stock|inc\.?|corp\.?|co\.?))',
            r'Symbols?:\s*([^.]*)'
        ]
        
        for pattern in patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    match = match[0]
                symbols.update(re.findall(r'[A-Z]{1,5}', match))
        
        false_positives = {'CEO', 'CFO', 'THE', 'A', 'I', 'API', 'AI', 'USA', 'PM', 'AM'}
        return sorted(list(symbols - false_positives))
        
    def collect_historical_news(self, start_date, end_date):
        try:
            # First verify we can get news providers
            providers = self.ib.reqNewsProviders()
            print("Available news providers:")
            for p in providers:
                print(f"- {p.code}: {p.name}")
            
            # Create Benzinga news contract
            news_contract = Contract()
            news_contract.symbol = "BZ:BZ_ALL"
            news_contract.secType = "NEWS"
            news_contract.exchange = "BZ"
            
            # Get contract details
            details = self.ib.reqContractDetails(news_contract)
            if not details:
                print("No contract details found")
                return pd.DataFrame()
                
            conId = details[0].contract.conId
            print(f"\nGot contract ID: {conId}")
            
            # Request historical news with retry
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    print(f"\nRequesting historical news (attempt {attempt + 1})...")
                    news = self.ib.reqHistoricalNews(
                        conId,
                        "BZ",
                        start_date,
                        end_date,
                        300,
                        []
                    )
                    
                    if news:
                        break
                    else:
                        print("No news data received, retrying...")
                        time.sleep(2)  # Wait before retry
                        
                except Exception as e:
                    print(f"Attempt {attempt + 1} failed: {str(e)}")
                    time.sleep(2)
            
            if not news:
                print("Failed to get historical news after all attempts")
                return pd.DataFrame()
            
            print(f"\nReceived {len(news)} news items")
            
            # Process each news item
            for n in news:
                try:
                    article = self.ib.reqNewsArticle("BZ", n.articleId)
                    symbols = self.extract_symbols(article.articleText)
                    
                    news_item = {
                        'timestamp': n.time,
                        'headline': n.headline,
                        'article_id': n.articleId,
                        'article_text': article.articleText,
                        'symbols': ', '.join(symbols),
                        'symbol_count': len(symbols),
                        'provider': "BZ",
                    }
                    
                    self.news_data.append(news_item)
                    print(f"Processed: {n.articleId} - Symbols: {symbols}")
                    time.sleep(0.1)  # Rate limiting
                    
                except Exception as e:
                    print(f"Error processing {n.articleId}: {str(e)}")
            
            # Create DataFrame
            df = pd.DataFrame(self.news_data)
            if not df.empty:
                df['timestamp'] = pd.to_datetime(df['timestamp'])
                df = df.sort_values('timestamp', ascending=False)
            return df
            
        except Exception as e:
            print(f"Error in collection process: {str(e)}")
            return pd.DataFrame()

# Usage
collector = NewsCollector()
collector.connect()

try:
    # Get historical news
    df = collector.collect_historical_news(
        start_date="2024-01-20 00:00:00",  # Added time component
        end_date="2024-01-21 23:59:59"     # Added time component
    )
    
    if not df.empty:
        # Save to CSV
        df.to_csv('benzinga_news.csv', index=False)
        print("\nSample of collected news:")
        print(df[['timestamp', 'headline', 'symbols', 'symbol_count']].head())
    else:
        print("\nNo data collected")
    
finally:
    collector.ib.disconnect()

Available news providers:
- BRFG: Briefing.com General Market Columns
- BRFUPDN: Briefing.com Analyst Actions
- BZ: Benzinga
- DJ-N: Dow Jones News Service
- DJ-RT: Dow Jones Trader News
- DJ-RTA: Dow Jones Real-Time News Asia Pacific
- DJ-RTE: Dow Jones Real-Time News Europe
- DJ-RTG: Dow Jones Real-Time News Global
- DJNL: Dow Jones Newsletters

Got contract ID: 137116869

Requesting historical news (attempt 1)...


reqHistoricalNewsAsync: Timeout


No news data received, retrying...

Requesting historical news (attempt 2)...

Received 300 news items
Processed: BZ$1751ea67 - Symbols: ['T']
Processed: BZ$1751ea66 - Symbols: ['SUV']
Processed: BZ$1751ea61 - Symbols: ['N', 'S', 'T']
Processed: BZ$1751ea60 - Symbols: ['T']
Processed: BZ$1751ea5f - Symbols: ['M', 'T']
Processed: BZ$1751ea5e - Symbols: ['BYD', 'T']
Processed: BZ$1751ea5d - Symbols: ['D', 'F', 'T']
Processed: BZ$1751ea5c - Symbols: ['T']
Processed: BZ$1751ea5b - Symbols: ['O', 'T']
Processed: BZ$1751ea5a - Symbols: []
Processed: BZ$1751ea59 - Symbols: []
Processed: BZ$1751ea57 - Symbols: []
Processed: BZ$1751ea56 - Symbols: []
Processed: BZ$1751ea54 - Symbols: []
Processed: BZ$1751ea4d - Symbols: []
Processed: BZ$1751ea4a - Symbols: ['F']
Processed: BZ$1751ea48 - Symbols: []
Processed: BZ$1751ea3f - Symbols: ['T']
Processed: BZ$1751ea38 - Symbols: []
Processed: BZ$1751ea37 - Symbols: ['T']
Processed: BZ$1751ea35 - Symbols: ['T']
Processed: BZ$1751ea34 - Symbols: ['T']
Pr

In [27]:
len(df)

300

In [35]:
from ib_insync import *
import pandas as pd
from datetime import datetime
import time
import re
from bs4 import BeautifulSoup

class NewsCollector:
    def __init__(self):
        self.ib = IB()
        self.news_data = []
        
    def connect(self):
        self.ib.connect('127.0.0.1', 7496, clientId=456)
        print("Connected to Live Trading")
        
    def extract_tickers_from_html(self, html_text):
        """Extract tickers from Benzinga HTML article text"""
        soup = BeautifulSoup(html_text, 'html.parser')
        tickers = []
        
        # Find all ticker links
        ticker_links = soup.find_all('a', class_='ticker')
        for link in ticker_links:
            ticker = link.text.strip()
            if ticker:
                tickers.append(ticker)
                
        return list(set(tickers))  # Remove duplicates
        
    def collect_historical_news(self, start_date, end_date):
        try:
            # Create BroadTape News contract
            news_contract = Contract()
            news_contract.symbol = "BZ:BZ_ALL"
            news_contract.secType = "NEWS"
            news_contract.exchange = "BZ"
            
            details = self.ib.reqContractDetails(news_contract)
            if not details:
                print("No contract details found")
                return pd.DataFrame()
                
            conId = details[0].contract.conId
            print(f"\nGot Benzinga ALL NEWS contract ID: {conId}")
            
            print(f"\nRequesting historical news from {start_date} to {end_date}...")
            news = self.ib.reqHistoricalNews(
                conId,
                "BZ",
                start_date,
                end_date,
                300,
                []
            )
            
            if not news:
                print("No news data received")
                return pd.DataFrame()
            
            print(f"\nReceived {len(news)} news items")
            
            for n in news:
                try:
                    article = self.ib.reqNewsArticle("BZ", n.articleId)
                    
                    # Extract tickers from HTML
                    tickers = self.extract_tickers_from_html(article.articleText)
                    
                    news_item = {
                        'timestamp': n.time,
                        'headline': n.headline,
                        'article_id': n.articleId,
                        'tickers': ', '.join(tickers),
                        'ticker_count': len(tickers),
                        'provider': n.providerCode,
                        'article_text': BeautifulSoup(article.articleText, 'html.parser').get_text()  # Clean text
                    }
                    
                    self.news_data.append(news_item)
                    print(f"Processed: {n.articleId} - Found tickers: {tickers}")
                    time.sleep(0.1)
                    
                except Exception as e:
                    print(f"Error processing {n.articleId}: {str(e)}")
            
            df = pd.DataFrame(self.news_data)
            if not df.empty:
                df['timestamp'] = pd.to_datetime(df['timestamp'])
                df = df.sort_values('timestamp', ascending=False)
                
                print("\nData Summary:")
                print(f"Total news items: {len(df)}")
                print(f"Unique tickers found: {len(set(','.join(df['tickers'].fillna('')).split(', ')))}")
                print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
            return df
            
        except Exception as e:
            print(f"Error in collection process: {str(e)}")
            return pd.DataFrame()

# Usage
collector = NewsCollector()
collector.connect()

try:
    df = collector.collect_historical_news(
        start_date="2024-01-23 00:00:00",
        end_date="2024-01-24 00:00:00"
    )
    
    if not df.empty:
        df.to_csv('benzinga_all_news.csv', index=False)
        print("\nSample of collected news:")
        print(df[['timestamp', 'headline', 'tickers', 'ticker_count']].head())
        
        all_tickers = set(','.join(df['tickers'].fillna('')).split(', '))
        print("\nUnique tickers found:", sorted(list(all_tickers)))
    else:
        print("\nNo data collected")
    
finally:
    collector.ib.disconnect()

Connected to Live Trading

Got Benzinga ALL NEWS contract ID: 137116869

Requesting historical news from 2024-01-23 00:00:00 to 2024-01-24 00:00:00...

Received 300 news items
Processed: BZ$1751ee7b - Found tickers: ['AAPL']
Processed: BZ$1751f2a1 - Found tickers: []
Processed: BZ$1751ee75 - Found tickers: ['BA', 'UAL']
Processed: BZ$1751f29f - Found tickers: ['ARVL']
Processed: BZ$1751ee73 - Found tickers: []
Processed: BZ$1751ee72 - Found tickers: ['ARKW', 'BITO', 'ARKB', 'MSFT', 'META', 'BTC']
Processed: BZ$1751ee6b - Found tickers: []
Processed: BZ$1751ee6a - Found tickers: ['LCID']
Processed: BZ$1751ee69 - Found tickers: ['AAPL']
Processed: BZ$1751ee67 - Found tickers: ['TSLA', 'LLY', 'HTZ', 'AMZN', 'GOOG', 'GOOGL', 'NVDA', 'META', 'MSFT', 'NVO', 'GM', 'AAPL']
Processed: BZ$1751ee66 - Found tickers: []
Processed: BZ$1751ee65 - Found tickers: []
Processed: BZ$1751ee64 - Found tickers: ['TCNNF']
Processed: BZ$1751ee63 - Found tickers: ['TSLA', 'NFLX', 'UAL', 'ADM', 'DWAC']
Processed

/var/folders/jc/lcqcq2_942v3dvq5dz2hlv3w0000gn/T/ipykernel_1400/2908819950.py:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, 'html.parser')
/var/folders/jc/lcqcq2_942v3dvq5dz2hlv3w0000gn/T/ipykernel_1400/2908819950.py:77: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  'article_text': BeautifulSoup(article.articleText, 'html.parser').get_text()  # Clean text


Processed: BZ$1751ee1e - Found tickers: []
Processed: BZ$1751ee1d - Found tickers: []
Processed: BZ$1751ee1c - Found tickers: ['NWBI']
Processed: BZ$1751ee1b - Found tickers: ['ZION']
Processed: BZ$1751ee1a - Found tickers: []
Processed: BZ$1751ee19 - Found tickers: ['PRK']
Processed: BZ$1751ee18 - Found tickers: []
Processed: BZ$1751ee17 - Found tickers: []
Processed: BZ$1751ee16 - Found tickers: []
Processed: BZ$1751ee15 - Found tickers: ['EFSC']
Processed: BZ$1751ee14 - Found tickers: ['KARO']
Processed: BZ$1751ee13 - Found tickers: ['DNTH']
Processed: BZ$1751ee12 - Found tickers: []
Processed: BZ$1751ee11 - Found tickers: ['CFB']
Processed: BZ$1751ee10 - Found tickers: ['EFSC']
Processed: BZ$1751ee0f - Found tickers: []
Processed: BZ$1751ee0e - Found tickers: ['LQR']
Processed: BZ$1751ee0d - Found tickers: ['PATH']
Processed: BZ$1751ee0c - Found tickers: []
Processed: BZ$1751ee0b - Found tickers: []
Processed: BZ$1751ee0a - Found tickers: []
Processed: BZ$1751ee09 - Found tickers: 

In [36]:
df

,timestamp,headline,article_id,tickers,ticker_count,provider,article_text
0,2024-01-23 04:59:36,{A:800015:L:en:K:n/a:C:0.40457791090011597}App...,BZ$1751ee7b,AAPL,1,BZ,Apple Inc. (NASDAQ:AAPL) has paid a hefty fine...
1,2024-01-23 04:54:32,{A:800015:L:en:K:n/a:C:0.9907137751579285}UAW ...,BZ$1751f2a1,,0,BZ,United Auto Workers (UAW) President Shawn Fain...
2,2024-01-23 04:52:24,{A:800015:L:en:K:n/a:C:0.38717857003211975}Uni...,BZ$1751ee75,"BA, UAL",2,BZ,"In a recent development, United Airlines Holdi..."
3,2024-01-23 04:48:28,{A:800015:L:en:K:n/a:C:0.8803068995475769}Elec...,BZ$1751f29f,ARVL,1,BZ,Electric vehicle manufacturer Arrival SA (NASD...
4,2024-01-23 04:27:38,{A:800015:L:en:K:0.92:C:0.9215885996818542}Mar...,BZ$1751ee73,,0,BZ,Rep. Marjorie Taylor Greene (R-Ga.) is set to ...
...,...,...,...,...,...,...,...
295,2024-01-22 17:15:49,{A:800015:L:en:K:n/a:C:0.918082594871521}A Clo...,BZ$1751ed37,ORCL,1,BZ,High-rolling investors have positioned themsel...
296,2024-01-22 17:15:43,{A:800015:L:en:K:n/a:C:0.9219000935554504}Behi...,BZ$1751ed36,HOOD,1,BZ,Financial giants have made a conspicuous bulli...
297,2024-01-22 17:15:38,{A:800015:L:en:K:n/a:C:0.7656286954879761}Mark...,BZ$1751ed35,MA,1,BZ,Whales with a lot of money to spend have taken...
298,2024-01-22 17:15:14,{A:800015:L:en:K:0.93:C:0.9291548132896423}!Co...,BZ$1751ed39,CMA,1,BZ,Shares of Comerica Incorporated (NYSE:CMA) fel...


In [77]:
from ib_insync import *
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import re

class NewsCollector:
    def __init__(self):
        self.ib = IB()
        self.news_data = []
        self.client_id = random.randint(1000, 9999)
        if self.ib.isConnected():
            self.ib.disconnect()
        
    def connect(self):
        self.ib.connect('127.0.0.1', 7496, clientId=self.client_id)
        print("Connected to Live Trading")
        
    def extract_tickers_from_html(self, html_text, headline):
        """Extract tickers using multiple methods"""
        tickers = set()
        
        # Method 1: Extract from HTML ticker links
        soup = BeautifulSoup(html_text, 'html.parser')
        ticker_links = soup.find_all('a', class_='ticker')
        for link in ticker_links:
            ticker = link.text.strip()
            if ticker and len(ticker) <= 5:
                tickers.add(ticker)
        
        # Method 2: Look for stock symbols in parentheses
        text = soup.get_text()
        parentheses_matches = re.findall(r'\((?:NYSE|NASDAQ):\s*([A-Z]{1,5})\)', text)
        tickers.update(parentheses_matches)
        
        # Method 3: Look for cashtags
        cashtags = re.findall(r'\$([A-Z]{1,5})\b', text)
        tickers.update(cashtags)
        
        # Method 4: Check headline for tickers
        headline_tickers = re.findall(r'\((?:NYSE|NASDAQ):\s*([A-Z]{1,5})\)', headline)
        tickers.update(headline_tickers)
        
        return sorted(list(tickers))
        
    def collect_historical_news(self, start_date, end_date):
        try:
            news_contract = Contract()
            news_contract.symbol = "BZ:BZ_ALL"
            news_contract.secType = "NEWS"
            news_contract.exchange = "BZ"
            
            details = self.ib.reqContractDetails(news_contract)
            if not details:
                print("No contract details found")
                return pd.DataFrame()
                
            conId = details[0].contract.conId
            print(f"\nGot Benzinga ALL NEWS contract ID: {conId}")
            
            news = self.ib.reqHistoricalNews(
                conId,
                "BZ",
                start_date,
                end_date,
                300,
                []
            )
            
            if not news:
                print("No news data received")
                return pd.DataFrame()
            
            print(f"\nReceived {len(news)} news items")
            
            for n in news:
                try:
                    article = self.ib.reqNewsArticle("BZ", n.articleId)
                    tickers = self.extract_tickers_from_html(article.articleText, n.headline)
                    
                    news_item = {
                        'timestamp': n.time,
                        'headline': n.headline,
                        'article_id': n.articleId,
                        'tickers': ', '.join(tickers),
                        'ticker_count': len(tickers),
                        'provider': n.providerCode,
                        'article_text': BeautifulSoup(article.articleText, 'html.parser').get_text()
                    }
                    
                    self.news_data.append(news_item)
                    
                except Exception as e:
                    print(f"Error processing {n.articleId}: {str(e)}")
            
            df = pd.DataFrame(self.news_data)
            if not df.empty:
                df['timestamp'] = pd.to_datetime(df['timestamp'])
                df = df.sort_values('timestamp', ascending=False)
                
                # Analyze articles with no tickers
                no_ticker_articles = df[df['ticker_count'] == 0]
                print(f"\nFound {len(no_ticker_articles)} articles with no tickers")
                if not no_ticker_articles.empty:
                    print("\nSample of articles with no tickers:")
                    for _, row in no_ticker_articles.head().iterrows():
                        print(f"\nHeadline: {row['headline']}")
                        print(f"Article ID: {row['article_id']}")
                        print("-" * 50)
                
                # Print distribution
                print("\nTicker count distribution:")
                print(df['ticker_count'].value_counts().sort_index())
                
            return df
            
        except Exception as e:
            print(f"Error in collection process: {str(e)}")
            return pd.DataFrame()

# Usage
collector = NewsCollector()
collector.connect()

try:
    df = collector.collect_historical_news(
        start_date="2024-01-23 00:00:00",
        end_date="2024-01-24 00:00:00"
    )
    
    if not df.empty:
        # Save full dataset
        df.to_csv('benzinga_all_news.csv', index=False)
        
        # Save articles with no tickers separately for review
        no_tickers = df[df['ticker_count'] == 0]
        if not no_tickers.empty:
            no_tickers.to_csv('benzinga_no_tickers.csv', index=False)
            print("\nArticles with no tickers saved to 'benzinga_no_tickers.csv'")
    
finally:
    collector.ib.disconnect()

Connected to Live Trading

Got Benzinga ALL NEWS contract ID: 137116869

Received 300 news items


/var/folders/jc/lcqcq2_942v3dvq5dz2hlv3w0000gn/T/ipykernel_1400/2279531465.py:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, 'html.parser')
/var/folders/jc/lcqcq2_942v3dvq5dz2hlv3w0000gn/T/ipykernel_1400/2279531465.py:88: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  'article_text': BeautifulSoup(article.articleText, 'html.parser').get_text()



Found 89 articles with no tickers

Sample of articles with no tickers:

Headline: {A:800015:L:en:K:n/a:C:0.9907137751579285}UAW Endorsement For Joe Biden Not A Done Deal Yet; Union President Says Will Hold Formal Discussions To Decide
Article ID: BZ$1751f2a1
--------------------------------------------------

Headline: {A:800015:L:en:K:0.92:C:0.9215885996818542}Marjorie Taylor Greene Heads To New Hampshire To Support Trump Ahead Of Crucial Vote: 'Tomorrow's Victory Will Be Historic!
Article ID: BZ$1751ee73
--------------------------------------------------

Headline: {A:800015:L:en:K:-0.92:C:0.9196704030036926}China Weighs $278B Stock Market Rescue Package Amid Slump: Report
Article ID: BZ$1751ee6b
--------------------------------------------------

Headline: {A:800015:L:en:K:0.98:C:0.9750197529792786}!Trump's 10% Tariff Plan Could 'Shake Up Every Asset Class,' Top Economist Warns: 'Structurally Breaking The Global System By Hook Or By Crook'
Article ID: BZ$1751ee66
------------------

In [50]:
set([ticker for tickers in df.tickers.tolist() for ticker in tickers.split(', ') if ticker])

{'A',
 'AAL',
 'AAPL',
 'ABBV',
 'ABNB',
 'ABT',
 'ACET',
 'ACGL',
 'ACNNF',
 'ACRS',
 'ADAG',
 'ADM',
 'ADSK',
 'AFRM',
 'AFYA',
 'AGFY',
 'AGNC',
 'AGYS',
 'ALB',
 'ALLY',
 'ALTM',
 'AMD',
 'AMP',
 'AMPY',
 'AMZN',
 'AON',
 'AQB',
 'ARBK',
 'ARKB',
 'ARKW',
 'ARM',
 'ARRY',
 'ARVL',
 'ASML',
 'AUTL',
 'AXP',
 'AZTR',
 'BA',
 'BBLG',
 'BBRRF',
 'BCLI',
 'BFRG',
 'BIDU',
 'BIMI',
 'BITO',
 'BKNG',
 'BLDE',
 'BLKB',
 'BLOZF',
 'BMMJ',
 'BMY',
 'BTC',
 'BYND',
 'CABA',
 'CARA',
 'CBAT',
 'CBDY',
 'CBNK',
 'CCG',
 'CCTG',
 'CDE',
 'CELH',
 'CETY',
 'CFB',
 'CHH',
 'CHPT',
 'CHRS',
 'CHTR',
 'CINT',
 'CISS',
 'CLDI',
 'CLSH',
 'CMA',
 'CMCSA',
 'CMTL',
 'CNBX',
 'CNSP',
 'CNTMF',
 'CNVCF',
 'COMM',
 'COMS',
 'COST',
 'CRBP',
 'CRM',
 'CRTD',
 'CSX',
 'CTHR',
 'CURLF',
 'CVKD',
 'CVNA',
 'CVS',
 'CVX',
 'CXAI',
 'CXDO',
 'DCFC',
 'DCGO',
 'DELL',
 'DHI',
 'DIA',
 'DIS',
 'DKNG',
 'DNTH',
 'DNUT',
 'DOCU',
 'DOGZ',
 'DVN',
 'DWAC',
 'EAST',
 'ECDA',
 'ECO',
 'EFSC',
 'EIS',
 'EKSO',
 'ELF',


In [78]:
df.ticker_count.value_counts()

ticker_count
1     167
0      89
12     10
2       6
3       6
6       5
10      5
4       4
5       3
37      2
11      1
7       1
42      1
Name: count, dtype: int64

In [79]:
df.provider.unique()

array(['BZ'], dtype=object)

In [80]:
df.head(2)

,timestamp,headline,article_id,tickers,ticker_count,provider,article_text
0,2024-01-23 04:59:36,{A:800015:L:en:K:n/a:C:0.40457791090011597}Apple Settles $13.7M Fine With Russia's Antitrust Watchdog Over In-App Payment Tactics,BZ$1751ee7b,AAPL,1,BZ,"Apple Inc. (NASDAQ:AAPL) has paid a hefty fine of $13.7 million to Russia’s Federal Antimonopoly Service or FAS. The fine was imposed on the tech behemoth for its alleged abuse of dominant market position.\nWhat Happened: Apple settled the fine, equivalent to 1.2 billion roubles, on Jan. 19, revealed Russia’s FAS antitrust on Monday. The FAS accused Apple of leveraging its in-app payment system to give its own products an unfair advantage, reported Reuters. \nThe FAS confirmed that the fine had been paid and the amount has been transferred to Russia’s federal budget. Apple, however, has yet to comment on this development. The company had previously expressed disagreement with the FAS’s decision, the report noted.\nSee Also: Supreme Court Decision Shakes Up Apple’s App Store, Opens Door for Cheaper App Purchases\nLast year, Apple had paid approximately $12.1 million in another antitrust case, regarding alleged abuse of its dominance in the mobile apps market. The company had also suspended all sales in Russia and limited its Apple Pay service following the Ukraine conflict in February 2022.\nWhy It Matters: This penalty comes at a time when Apple has been facing backlash over its controversial App Store policies. \nApple has updated its App Store guidelines, allowing U.S. developers to incorporate links to external payment platforms. \nDespite this, developers still have to pay a percentage of the transaction to Apple. The fee has been reduced to 27% or 12% for developers in the App Store Small Business Program.\nHowever, critics and developers have described these changes as outrageous and predict potential legal disputes.\nPhoto via Shutterstock\nRead Next: Apple App Store Data Shows Meta Stole Christmas, Amazon Enjoys Strong Holiday Shopping Trends\nCheck out more of Benzinga’s Consumer Tech coverage by following this link.\nDisclaimer: This content was partially produced with the help of Benzinga Neuro and was reviewed and published by Benzinga editors."
1,2024-01-23 04:54:32,{A:800015:L:en:K:n/a:C:0.9907137751579285}UAW Endorsement For Joe Biden Not A Done Deal Yet; Union President Says Will Hold Formal Discussions To Decide,BZ$1751f2a1,,0,BZ,"United Auto Workers (UAW) President Shawn Fain reportedly said on Monday that the union is yet to decide on which presidential candidate to endorse, dashing hopes of an imminent endorsement of President Joe Biden.\nWhat Happened: Despite expressing criticism towards ex-President Donald Trump, Fain clarified that the UAW is yet to formalize its presidential endorsement, Reuters reported. Fain did not comment on reports of an upcoming address by President Biden to the union members in Washington during a legislative conference this week. However, the union president noted that Trump as a person is ‘contrary’ to everything the union stands for. \nNotably, Fain confirmed to Reuters that the UAW would be holding formal discussions regarding the endorsement. \nWhy It Matters: As Biden and Trump emerge as frontrunners for the Democratic and Republican parties, respectively, the endorsement from UAW could be a significant boost for Biden's campaign. \nThe union’s endorsement holds substantial political weight, given the union's significant influence in Michigan, a pivotal state in the 2024 presidential race. Biden narrowly won the state in the 2020 election. \nPresident Biden has been attempting to garner support from the UAW. He made history in September last year by becoming the first sitting U.S. president to join a UAW picket line during a major strike. \nHowever, in May, Fain explained the reason for not endorsing Biden for the second term as disagreement on his EV policies. Fain alleged that the government is allotting funds in billions to e

In [131]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta
import random
from bs4 import BeautifulSoup
import re

class BenzingaNewsCollector:
    def __init__(self):
        self.ib = IB()
        self.news_data = []
        
    def connect(self):
        try:
            client_id = random.randint(1000, 9999)
            if self.ib.isConnected():
                self.ib.disconnect()
            self.ib.connect('127.0.0.1', 7496, clientId=client_id)
            print(f"Connected to IBKR with client ID: {client_id}")
            self.ib.sleep(1)
            return True
        except Exception as e:
            print(f"Connection error: {str(e)}")
            return False
            
    def disconnect(self):
        if self.ib.isConnected():
            self.ib.disconnect()
            print("Disconnected from IBKR")

    def get_contract(self, symbol):
        """Get the contract for a given symbol."""
        try:
            contract = Stock(symbol, 'SMART', 'USD')
            self.ib.qualifyContracts(contract)
            return contract
        except Exception as e:
            print(f"Error getting contract for {symbol}: {str(e)}")
            return None

    def extract_tickers_from_html(self, html_text):
        """Extract stock tickers from HTML text."""
        tickers = set()
        if not html_text:
            return tickers
            
        soup = BeautifulSoup(html_text, 'html.parser')
        
        # Get tickers from links
        ticker_links = soup.find_all('a', class_='ticker')
        for link in ticker_links:
            ticker = link.text.strip()
            if ticker and len(ticker) <= 5:  # Most tickers are 5 or fewer characters
                tickers.add(ticker)
                
        return tickers

    def extract_clean_text(self, html_text):
        """Extract clean text from HTML."""
        if not html_text:
            return ""
            
        soup = BeautifulSoup(html_text, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
            
        # Get text and clean it
        text = soup.get_text(separator=' ')
        
        # Clean up whitespace
        text = ' '.join(text.split())
        
        return text

    def get_stock_news(self, symbol='AAPL', start_date=None, end_date=None):
        """
        Collect news articles for a given symbol within a date range.
        
        Args:
            symbol (str): Stock symbol (default: 'AAPL')
            start_date (str): Start date in 'YYYY-MM-DD' format
            end_date (str): End date in 'YYYY-MM-DD' format
        
        Returns:
            pd.DataFrame: DataFrame containing collected news articles
        """
        try:
            # Initialize dates
            if not start_date or not end_date:
                end_date = datetime.now()
                start_date = end_date - timedelta(days=5)
            else:
                start_date = pd.to_datetime(start_date)
                end_date = pd.to_datetime(end_date)
                
            # Ensure we're not getting future dates
            current_time = datetime.now()
            if end_date > current_time:
                end_date = current_time
            
            print(f"\nCollecting news from {start_date.date()} to {end_date.date()}")
            
            # Get contract ID
            contract = self.get_contract(symbol)
            if not contract:
                print(f"Error: Could not find contract for symbol {symbol}")
                return pd.DataFrame()
            
            # Get news in smaller time chunks to avoid the 50-article limit
            current_date = start_date
            all_news = []
            self.news_data = []  # Reset news data
            
            while current_date <= end_date:
                chunk_end = min(current_date + timedelta(days=1), end_date)
                
                try:
                    news = self.ib.reqHistoricalNews(
                        conId=contract.conId,
                        providerCodes='BZ',
                        startDateTime=current_date.strftime('%Y%m%d %H:%M:%S'),
                        endDateTime=chunk_end.strftime('%Y%m%d %H:%M:%S'),
                        totalResults=1000
                    )
                    
                    if news:
                        all_news.extend(news)
                        print(f"Found {len(news)} articles for {current_date.date()}")
                    else:
                        print(f"No articles found for {current_date.date()}")
                    
                except Exception as e:
                    print(f"Error fetching news for {current_date.date()}: {str(e)}")
                
                current_date = chunk_end + timedelta(seconds=1)
                self.ib.sleep(1)  # Avoid rate limiting
            
            print(f"\nTotal articles found: {len(all_news)}")
            
            # Process articles
            processed_articles = set()
            for n in all_news:
                try:
                    # Skip duplicates
                    if n.articleId in processed_articles:
                        continue
                    
                    # Validate article date
                    article_date = pd.to_datetime(n.time)
                    if article_date > current_time:
                        print(f"Skipping article with future date: {article_date}")
                        continue
                    
                    # Get full article text
                    article = self.ib.reqNewsArticle("BZ", n.articleId)
                    
                    # Clean the headline
                    clean_headline = re.sub(r'\{.*?\}', '', n.headline).strip()
                    
                    # Extract tickers
                    mentioned_tickers = self.extract_tickers_from_html(article.articleText)
                    
                    # Add to processed set
                    processed_articles.add(n.articleId)
                    
                    # Store the article data
                    self.news_data.append({
                        'timestamp': article_date,
                        'symbol': symbol,
                        'headline': clean_headline,
                        'article_id': n.articleId,
                        'provider': n.providerCode,
                        'article_text': self.extract_clean_text(article.articleText),
                        'mentioned_tickers': ', '.join(mentioned_tickers),
                        'ticker_count': len(mentioned_tickers)
                    })
                    
                    print(f"Processed article: {n.articleId} - Found {len(mentioned_tickers)} tickers")
                    self.ib.sleep(0.2)  # Avoid rate limiting
                    
                except Exception as e:
                    print(f"Error processing article {n.articleId}: {str(e)}")
            
            # Create DataFrame and process
            df = pd.DataFrame(self.news_data)
            if not df.empty:
                # Remove duplicates and sort
                df = df.drop_duplicates(subset=['article_id'])
                df['timestamp'] = pd.to_datetime(df['timestamp'])
                df = df.sort_values('timestamp', ascending=False)
                
                print("\nCollection Summary:")
                print(f"Total unique articles: {len(df)}")
                print(f"Date range: {df.timestamp.min()} to {df.timestamp.max()}")
                print(f"Total tickers mentioned: {df['ticker_count'].sum()}")
                
                # Validate date range
                if df.timestamp.max() > current_time:
                    print("\nWarning: Some articles have future dates!")
                
            return df
                
        except Exception as e:
            print(f"Error collecting news: {str(e)}")
            return pd.DataFrame()
        


collector = BenzingaNewsCollector()
try:
    if collector.connect():
        df = collector.get_stock_news(
            symbol='AAPL',
            start_date='2024-01-01',
            end_date='2024-01-08'
        )
        if not df.empty:
            print("\nSample of collected news:")
            print(df[['timestamp', 'headline', 'mentioned_tickers']].head())
            df.to_csv('aapl_news.csv', index=False)
            print("\nData saved to 'aapl_news.csv'")
    else:
        print("Failed to connect to IBKR")
finally:
    collector.disconnect()


Connected to IBKR with client ID: 4580

Found 300 articles for 2024-01-01
Found 300 articles for 2024-01-02
Found 300 articles for 2024-01-03
Found 300 articles for 2024-01-04
Found 300 articles for 2024-01-05
Found 300 articles for 2024-01-06
Found 300 articles for 2024-01-07

Total articles found: 2100
Processed article: BZ$19b6a9d9 - Found 3 tickers
Processed article: BZ$19b52918 - Found 8 tickers
Processed article: BZ$19b4d7c3 - Found 0 tickers
Processed article: BZ$19b47bcc - Found 4 tickers
Processed article: BZ$19b442e2 - Found 1 tickers
Processed article: BZ$19b41146 - Found 1 tickers
Processed article: BZ$19b40b85 - Found 9 tickers
Processed article: BZ$19b402de - Found 5 tickers
Processed article: BZ$19b3fda1 - Found 0 tickers
Processed article: BZ$19b3e08d - Found 7 tickers
Processed article: BZ$19b3d31e - Found 6 tickers
Processed article: BZ$19b3a6db - Found 1 tickers
Processed article: BZ$19b366d2 - Found 2 tickers
Processed article: BZ$19b342e6 - Found 2 tickers
Processe

timestamp	symbol	headline	article_id	provider	article_text	source	author	url	summary	extra_data	updated_time

In [132]:
df.ticker_count.value_counts()

ticker_count
1     92
10    34
0     29
2     27
3     23
5     18
8     16
4     14
6     11
7     10
9      5
11     5
13     4
12     3
14     2
16     2
21     1
17     1
23     1
18     1
15     1
Name: count, dtype: int64

In [133]:
df.timestamp.min(), df.timestamp.max()

(Timestamp('2024-11-26 10:08:56'), Timestamp('2025-01-08 11:35:11'))

In [134]:
df.head(2)

,timestamp,symbol,headline,article_id,provider,article_text,mentioned_tickers,ticker_count
0,2025-01-08 11:35:11,AAPL,"Dell Simplifies PC Lineup With Apple-Like Branding: XPS, Inspiron Brands Are Gone For Good",BZ$19b6a9d9,BZ,"On Monday, Dell Technologies Inc. (NYSE: DELL ) announced a major rebranding of its personal computers, aiming to boost sales by adopting a simplified naming strategy similar to Apple Inc. (NASDAQ: AAPL ). What Happened: The company has decided to retire long-standing product names like “XPS” and “Inspiron” in favor of a streamlined approach centered around the Dell name. The rebranding is part of Dell’s strategy to make its products more appealing and less confusing for consumers. The new lineup will feature three tiers: Dell , Dell Pro , and Dell Pro Max , echoing Apple’s naming conventions. See Also: Jensen Huang Unveils $3000 ‘Personal AI Supercomputer' The Size Of A Small Book: ‘It Runs The Entire Nvidia AI Stack' The rebranding is also part of a broader effort to encourage PC upgrades, with AI-optimized systems and the end of support for Microsoft Corp.’s (NASDAQ: MSFT ) Windows 10 as potential catalysts. Notably, Dell’s gaming brand, Alienware , will not be affected by the rebranding. The new Dell-branded devices will incorporate neural processing units to improve AI capabilities. Subscribe to the Benzinga Tech Trends newsletter to get all the latest tech developments delivered to your inbox. Why It Matters: In November 2024, Dell announced earnings of $2.15 per share for the third quarter, surpassing the analyst consensus estimate of $2.05. However, the company's quarterly revenue totaled $24.37 billion, falling short of the expected $24.65 billion, though it represented a growth from the $22.25 billion recorded during the same quarter last year. In the third quarter of 2024, Dell’s share of global PC shipments dropped to 15.7%, down from 16.7% in the previous quarter. The company remains the third-largest PC vendor worldwide, trailing behind Lenovo and HP, according to Statista. Price Action: Dell's stock fell by 2.72% on Tuesday, ending the day at $121.27. However, the stock bounced back in Wednesday's pre-market session, increasing by 1.3% to $122.85, according to Benzinga Pro data . Check out more of Benzinga's Consumer Tech coverage by following this link . Read Next: Nvidia Soars, AMD Stalls: AI Race Heats Up At CES 2025 Disclaimer: This content was partially produced with the help of AI tools and was reviewed and published by Benzinga editors. Photo courtesy: Shutterstock Copyright © Benzinga. All rights reserved. Write to editorial@benzinga.com with any questions about this content. Benzinga does not provide investment advice.","AAPL, DELL, MSFT",3
1,2025-01-07 21:47:40,AAPL,!3 Digital Infrastructure ETFs In Focus As Trump Reveals Billion-Dollar Data Center Construction Plan,BZ$19b52918,BZ,"President-elect Donald Trump announced on Tuesday that the Emirati real estate development firm DAMAC Properties will invest $20 billion to construct data centers in the U.S. to meet the growing needs of hyperscale customers. The race for dominance in artificial intelligence hit an inflection point as Microsoft (NASDAQ: MSFT ) revealed its $80 billion investment plan to build AI-driven data centers in fiscal 2025, with special regional focus on the U.S. These developments serve as a green light for investors looking to invest in this theme. Risk-averse investors who are interested in cashing in on this opportunity, but are reluctant to choose individual stocks can choose from a plethora of ETFs tied to data center and digital infrastructure stocks. Here we name 3 such ETFs, which carry stocks of digital infrastructure providers like Microsoft and are poised to mirror the gains of the digital infrastructure industry. See Also: Microsoft Pauses Construction On $3.3 Billion Data Center Expected To Be Used By ChatGPT-Parent OpenAI For Massive Super Computer Vanguard Information Technology ETF (NYSE: VGT ),

In [135]:
print(f"Length of df: {len(df)}")
print(f"Columns of df: {df.columns}")
print(f"News with no tickers: {df.symbol.isna().sum()}")

Length of df: 300
Columns of df: Index(['timestamp', 'symbol', 'headline', 'article_id', 'provider',
       'article_text', 'mentioned_tickers', 'ticker_count'],
      dtype='object')
News with no tickers: 0


In [124]:
df.iloc[0].headline

'{A:800015:L:en:K:0.54:C:0.5353515148162842}Dell Simplifies PC Lineup With Apple-Like Branding: XPS, Inspiron Brands Are Gone For Good'

In [58]:
# Display all rows and columns without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
        